# LangChain Cookbook 👨‍🍳👩‍🍳

In [1]:
!pip install accelerate==0.18.0
!pip install transformers==4.28.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.9 MB/s eta 0:00:00


In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset, DatasetDict

In [4]:
raw_dataset = load_dataset('ccdv/cnn_dailymail','3.0.0')

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [6]:
raw_dataset['train'][0]['article'][:200]

"It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours afte"

In [7]:
raw_dataset['train'].to_pandas()

,article,highlights,id
0,It's official: U.S. President Barack Obama wan...,Syrian official: Obama climbed to the top of t...,0001d1afc246a7964130f43ae940af6bc6c57f01
1,(CNN) -- Usain Bolt rounded off the world cham...,Usain Bolt wins third gold of world championsh...,0002095e55fcbd3a2f366d9bf92a95433dc305ef
2,"Kansas City, Missouri (CNN) -- The General Ser...",The employee in agency's Kansas City office is...,00027e965c8264c35cc1bc55556db388da82b07f
3,Los Angeles (CNN) -- A medical doctor in Vanco...,NEW: A Canadian doctor says she was part of a ...,0002c17436637c4fe1837c935c04de47adb18e9a
4,(CNN) -- Police arrested another teen Thursday...,Another arrest made in gang rape outside Calif...,0003ad6ef0c37534f80b55b4235108024b407f0b
...,...,...,...
287108,Tiger Woods’s frustration at the lamentable st...,"Woods said: ’Guys, give me a little f***ing sp...",fffdfb56fdf1a12d364562cc2b9b1d4de7481dee
287109,By . Mark Duell . Last updated at 4:07 PM on 2...,13 sailors died in 1804 after explosives ship ...,fffeecb8690b85de8c3faed80adbc7a978f9ae2a
287110,"Suicide: Troll victim Hannah Smith, 14, killed...",Hannah Smith's father says Ask.fm's safety cha...,ffff5231e4c71544bc6c97015cdb16c60e42b3f4
287111,By . Victoria Woollaston and Mark Prigg . PUBL...,A test version of Windows 8.1 is available to ...,ffff924b14a8d82058b6c1c5368ff1113c1632af


In [8]:
sampled_dataset = DatasetDict(
  {
      'train': raw_dataset['train'].select(range(5000)).shuffle(),
      'valid': raw_dataset['test'].select(range(5000)).shuffle()
  }
)

In [9]:
raw_dataset['train'][0]

{'article': 'It\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It\'s a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but bec

## tokenizer

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)

In [11]:
def get_training_corpus(ds):
  return(
      ds[i:i+1000]['article'] for i in range(0,len(ds),1000)
  )

In [12]:
training_corpus = get_training_corpus(raw_dataset['train'])
training_corpus

<generator object get_training_corpus.<locals>.<genexpr> at 0x7924b6d94ba0>

In [13]:
%%time

tokenizer = tokenizer.train_new_from_iterator(training_corpus,vocab_size = 50257)

CPU times: user 13min 15s, sys: 2.98 s, total: 13min 18s
Wall time: 8min 9s


In [14]:
sample_text = "It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria"

tokenizer.tokenize(sample_text)

['It',
 "'s",
 'Ġofficial',
 ':',
 'ĠU',
 '.',
 'S',
 '.',
 'ĠPresident',
 'ĠBarack',
 'ĠObama',
 'Ġwants',
 'Ġlawmakers',
 'Ġto',
 'Ġweigh',
 'Ġin',
 'Ġon',
 'Ġwhether',
 'Ġto',
 'Ġuse',
 'Ġmilitary',
 'Ġforce',
 'Ġin',
 'ĠSyria']

In [15]:
tokenizer(sample_text, return_length=True)

{'input_ids': [868, 345, 1061, 26, 458, 14, 51, 14, 1497, 4149, 1288, 2880, 8505, 280, 6284, 285, 316, 1714, 280, 1321, 1681, 2692, 285, 2730], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'length': [24]}

In [16]:
context_length = 128

def tokenize(batch):
    outputs = tokenizer(
        batch['article'],
        max_length=context_length,
        truncation=True,
        return_overflowing_tokens=True,
        return_length=True
    )

    input_batch = []
    for length, input_ids in zip(outputs['length'], outputs['input_ids']):
        if length==context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}

In [17]:
tokenized_datasets = sampled_dataset.map(tokenize, batched=True, remove_columns=raw_dataset['train'].column_names)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [18]:
tokenized_datasets['train']

Dataset({
    features: ['input_ids'],
    num_rows: 29045
})

In [19]:
from transformers import LlamaConfig

configuration = LlamaConfig()

configuration

LlamaConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "tie_word_embeddings": false,
  "transformers_version": "4.28.0",
  "use_cache": true,
  "vocab_size": 32000
}

In [20]:
tokenizer.bos_token_id, tokenizer.eos_token_id, tokenizer.vocab_size

(0, 0, 50257)

In [21]:
configuration = LlamaConfig(**{
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 1376,
  "max_position_embeddings": 128,
  "model_type": "llama",
  "num_attention_heads": 4,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "tie_word_embeddings": False,
  "transformers_version": "4.28.0",
  "use_cache": True,
  "vocab_size": 50257
})

In [22]:
from transformers import LlamaForCausalLM

model = LlamaForCausalLM(configuration)
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(50257, 512, padding_idx=0)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=512, out_features=512, bias=False)
          (k_proj): Linear(in_features=512, out_features=512, bias=False)
          (v_proj): Linear(in_features=512, out_features=512, bias=False)
          (o_proj): Linear(in_features=512, out_features=512, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=512, out_features=1376, bias=False)
          (down_proj): Linear(in_features=1376, out_features=512, bias=False)
          (up_proj): Linear(in_features=512, out_features=1376, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_he

In [23]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [24]:
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(50257, 512, padding_idx=0)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=512, out_features=512, bias=False)
          (k_proj): Linear(in_features=512, out_features=512, bias=False)
          (v_proj): Linear(in_features=512, out_features=512, bias=False)
          (o_proj): Linear(in_features=512, out_features=512, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=512, out_features=1376, bias=False)
          (down_proj): Linear(in_features=1376, out_features=512, bias=False)
          (up_proj): Linear(in_features=512, out_features=1376, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_he

In [25]:
prompt = "It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in "

inputs = tokenizer(prompt, return_tensors='pt')
inputs.to(device)

generate_ids = model.generate(inputs.input_ids, max_length=50)
generate_ids

tensor([[  868,   345,  1061,    26,   458,    14,    51,    14,  1497,  4149,
          1288,  2880,  8505,   280,  6284,   285,   316,  1714,   280,  1321,
          1681,  2692,   285,   221, 46510, 15180, 20580, 49255, 20580, 39110,
         39110, 39110, 39110, 39110, 39110, 39110, 39110, 39110, 39110, 39110,
         39110, 39110, 39110, 39110, 39110, 39110, 39110, 39110, 39110, 39110]],
       device='cuda:0')

In [26]:
#?model.generate

In [27]:
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

"It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in irms Locanglingambanganangling Levant Levant Levant Levant Levant Levant Levant Levant Levant Levant Levant Levant Levant Levant Levant Levant Levant Levant Levant Levant Levant"

In [28]:
##train model

In [29]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [30]:
out = data_collator([tokenized_datasets['train'][i] for i in range(3)])

for key in out:
    print(f"{key}: {out[key].shape}")

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


input_ids: torch.Size([3, 128])
attention_mask: torch.Size([3, 128])
labels: torch.Size([3, 128])


In [31]:
out['input_ids'][0][:20], out['attention_mask'][0][:20], out['labels'][0][:20]

(tensor([45843, 14247,    12, 19960,   527,  1356,     9,   630,  1132,  6405,
         14247,    12,   262,  2643,  1333, 41212, 12696,   400,   262,  3724]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 tensor([45843, 14247,    12, 19960,   527,  1356,     9,   630,  1132,  6405,
         14247,    12,   262,  2643,  1333, 41212, 12696,   400,   262,  3724]))

In [32]:
from transformers import TrainingArguments

In [33]:
batch_size = 32
logging_steps = 1000
num_epochs = 10
#args = TrainingArguments(output_dir='newsllama')

args = TrainingArguments(
  output_dir='newsllama',
  per_device_train_batch_size=batch_size,
  per_device_eval_batch_size=batch_size,
  evaluation_strategy='steps',
  eval_steps=logging_steps,
  logging_steps=logging_steps,
  save_steps=logging_steps,
  gradient_accumulation_steps=8,
  num_train_epochs= num_epochs,
  weight_decay=0.1,
  warmup_steps=logging_steps,
  lr_scheduler_type='cosine',
  learning_rate=5e-4,
  fp16=False,
  push_to_hub=False
)

In [34]:
from transformers import Trainer

In [35]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['valid']
)

In [36]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
1000,5.909100,6.028381


TrainOutput(global_step=1130, training_loss=5.656767259445866, metrics={'train_runtime': 3253.5614, 'train_samples_per_second': 89.271, 'train_steps_per_second': 0.347, 'total_flos': 8524883389317120.0, 'train_loss': 5.656767259445866, 'epoch': 9.96})

In [37]:
prompt = "It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in "

inputs = tokenizer(prompt, return_tensors='pt')
inputs.to(device)

generate_ids = model.generate(inputs.input_ids, max_length=50)
generate_ids

tensor([[ 868,  345, 1061,   26,  458,   14,   51,   14, 1497, 4149, 1288, 2880,
         8505,  280, 6284,  285,  316, 1714,  280, 1321, 1681, 2692,  285,  221,
         2730,   14, 1288,  405,  476,  285,  262, 1500,   14,  548,  405,  361,
          337,  306,  405,  413,   65,  960,   13, 4332, 1358,  280,  262,  595,
          290, 2265]], device='cuda:0')

In [38]:
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'It\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in  Syria. Obama has been in the past. He has said that he has "a long-term support to the people of Iraq'

In [39]:
aaa = 1

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [43]:
! mkdir /content/drive/MyDrive/test

In [44]:
import os
os.getcwd()

'/content'

In [45]:
! mkdir /content/drive/MyDrive/test/result

In [46]:
! cp -r /content/newsllama /content/drive/MyDrive/test/result

In [47]:
model.save_pretrained('daily_llama_0612')

In [48]:
tokenizer.save_pretrained('daily_tokenizer_0612')

('daily_tokenizer_0612/tokenizer_config.json',
 'daily_tokenizer_0612/special_tokens_map.json',
 'daily_tokenizer_0612/vocab.json',
 'daily_tokenizer_0612/merges.txt',
 'daily_tokenizer_0612/added_tokens.json',
 'daily_tokenizer_0612/tokenizer.json')

In [49]:
! mkdir /content/drive/MyDrive/test/backup

In [50]:
! cp -r /content/daily_llama_0612 /content/drive/MyDrive/test/backup

In [51]:
! cp -r /content/daily_tokenizer_0612 /content/drive/MyDrive/test/backup

In [52]:
prompt = """Shall I compare thee to a summer’s day?
Thou art more lovely and more temperate:
Rough winds do shake the"""
inputs = tokenizer(prompt, return_tensors="pt")
inputs.to("cuda:0")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=50)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

"Shall I compare thee to a summer’s day?\nThou art more lovely and more temperate:\nRough winds do shake the story. It's not the only time I see that the children are in the hands of their"

In [53]:
prompt = """As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in"""
inputs = tokenizer(prompt, return_tensors="pt")
inputs.to("cuda:0")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=50)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in making the show more difficult. They had a lot of fun and fun.'